Assignment 5: Ben Hawks - z1849064

Part 1: Bag of Words Model



In [0]:
%matplotlib inline
!python3 -m spacy download en_core_web_lg
import spacy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl



train_data_url='https://raw.githubusercontent.com/ben-hawks/CSCI480J-AppliedMachineLearning/master/Assignment%205/Data/rscience-popular-comment-removal/reddit_200k_train.csv'
train_data = pd.read_csv(train_data_url,delimiter=",",encoding = 'ISO-8859-1')
train_data_X = train_data.body
train_data_Y = train_data.REMOVED


test_data_url='https://raw.githubusercontent.com/ben-hawks/CSCI480J-AppliedMachineLearning/master/Assignment%205/Data/rscience-popular-comment-removal/reddit_200k_test.csv'
test_data = pd.read_csv(test_data_url,delimiter=",",encoding = 'ISO-8859-1')
test_data_x = test_data.body
test_data_y = test_data.REMOVED

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV

vect_bow = CountVectorizer(encoding = 'ISO-8859-1',min_df=5)#Min_df = 5 arbitrarily, mostly just to reduce total features to real words and not just random strings 
x_train = vect_bow.fit_transform(train_data_X)
x_test_fit = vect_bow.transform(test_data_x)

In [0]:
print(vect_bow.get_feature_names())

['00', '000', '00001', '0001', '0001f3fb', '0001f3fc', '0001f44c', '0001f44d', '0001f44f', '0001f601', '0001f602', '0001f605', '0001f606', '0001f609', '0001f60e', '0001f60f', '0001f610', '0001f611', '0001f612', '0001f614', '0001f615', '0001f622', '0001f624', '0001f629', '0001f62d', '0001f631', '0001f643', '0001f644', '0001f914', '0001f923', '0001f926', '0001f937', '0004', '0006', '0009', '000x', '000â', '001', '0012', '002', '0022', '00224499', '0024', '003', '0031', '0039', '004', '005', '006', '007', '008', '0083', '009', '0092', '00am', '00pm', '00s', '00â', '01', '010', '0106', '011', '012', '013', '014', '0147', '015', '016', '017', '018', '019', '0190', '0191', '01926230490520232', '0195', '01â', '02', '020', '022', '023', '024', '025', '026', '027', '028', '029', '0296', '02â', '03', '030', '031', '032', '034', '035c', '0361', '0366', '038', '039', '04', '040', '041', '042', '043', '0430', '046', '047', '049', '05', '051', '055', '06', '065', '06â', '07', '072', '0747', '07â', '

In [0]:
CV_BOW = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[100],scoring='f1', verbose=1, cv=10, n_jobs=200).fit(x_train, train_data_Y)

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done   7 out of  10 | elapsed:  4.2min remaining:  1.8min
[Parallel(n_jobs=200)]: Done  10 out of  10 | elapsed:  4.9min finished


In [0]:
bow_score = CV_BOW.score(x_test_fit,test_data_y)
print ("Score for baseline Bag of Words (min_df = 5): " + str(bow_score))
print(CV_BOW.scores_)

Score for baseline Bag of Words (min_df = 5): 0.576097344926455
{True: array([[0.55845455],
       [0.56704981],
       [0.57398897],
       [0.5770108 ],
       [0.57252553],
       [0.55843959],
       [0.57219251],
       [0.5672582 ],
       [0.56759026],
       [0.57337936]])}


It didn't perform horribly to be honest, just not fantastically either. About what i expected for a baseline approach

Part 2: Optimizing Baseline model

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
# We have a big dataset, so lets make some abritrary decisison for the sake of training time (the above model took ~5 mins @n_jobs =200)
# Character Ngrams could be useful, but would create an *Obscene* amount of features,so lets ignore those for now
CV_NGram = CountVectorizer(encoding = 'ISO-8859-1',
                         min_df=5,
                         ngram_range=(1,3), #1-3 to begin with for now 
                         )
x_train_ng = CV_NGram.fit_transform(train_data_X)
x_test_ng = CV_NGram.transform(test_data_x)



In [0]:
TFID_NGram = TfidfVectorizer(encoding = 'ISO-8859-1', # Try TFID and Add some stop words, too!
                            ngram_range=(1, 3),
                            min_df=5,
                            stop_words='english')
x_train_tfid = TFID_NGram.fit_transform(train_data_X)
x_test_fit_tfid = TFID_NGram.transform(test_data_x)

In [0]:
ng_bow = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[100],scoring='f1', verbose=1, cv=10, n_jobs=200).fit(x_train_ng, train_data_Y)
ng_bow_score = ng_bow.score(x_test_ng,test_data_y)
print ("Score for NGram (1,3) Bag of Words (min_df = 5): " + str(ng_bow_score))
print(ng_bow.scores_)

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=200)]: Done   7 out of  10 | elapsed: 40.4min remaining: 17.3min
[Parallel(n_jobs=200)]: Done  10 out of  10 | elapsed: 49.5min finished


Score for NGram (1,3) Bag of Words (min_df = 5): 0.548954819760905
{True: array([[0.54759179],
       [0.54644975],
       [0.5518018 ],
       [0.5502439 ],
       [0.54436433],
       [0.54461375],
       [0.55542961],
       [0.55230126],
       [0.54635787],
       [0.5474818 ]])}


In [0]:
tfid_bow = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[100],scoring='f1', verbose=1, cv=10, n_jobs=200).fit(x_train_tfid, train_data_Y)


In [0]:
tfid_bow_score = tfid_bow.score(x_test_fit_tfid,test_data_y)
print ("Score for TFID/NGram (1,3) Bag of Words (min_df = 5): " + str(tfid_bow_score))
print(tfid_bow.scores_)

Score for TFID/NGram (1,3) Bag of Words (min_df = 5): 0.5357086050580564
{True: array([[0.53172703],
       [0.53534898],
       [0.5394272 ],
       [0.53418835],
       [0.5438701 ],
       [0.52794151],
       [0.53745684],
       [0.54062523],
       [0.53531599],
       [0.53486798]])}


Seems like the regularization of C=100 didn't do much, since this one is really short to train( vs the 1hr for ngram without tfid!!) Try a few different C Values to see whats working better

In [0]:
tfid_bow_CV = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[10,100,1000],scoring='f1', verbose=1, cv=10, n_jobs=200).fit(x_train_tfid, train_data_Y)


[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=200)]: Done   7 out of  10 | elapsed: 17.6min remaining:  7.6min
[Parallel(n_jobs=200)]: Done  10 out of  10 | elapsed: 18.9min finished


In [0]:
tfid_bow_CV_score = tfid_bow_CV.score(x_test_fit_tfid,test_data_y)
print ("Score for TFID/NGram (1,3) Bag of Words (min_df = 5): " + str(tfid_bow_CV_score))
print(tfid_bow_CV.scores_)

In [0]:
tfid_bow_CV2 = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[0.0001,0.001,0.01,0.1,1],scoring='f1', verbose=1, cv=10, n_jobs=200).fit(x_train_tfid, train_data_Y)
tfid_bow_CV2_score = tfid_bow_CV2.score(x_test_fit_tfid,test_data_y)
print ("Score for TFID/NGram (1,3) Bag of Words (min_df = 5): " + str(tfid_bow_CV2_score))
print(tfid_bow_CV2.scores_)

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done   7 out of  10 | elapsed:   49.2s remaining:   21.1s
[Parallel(n_jobs=200)]: Done  10 out of  10 | elapsed:   49.9s finished


Score for TFID/NGram (1,3) Bag of Words (min_df = 5): 0.5357086050580564
{True: array([[0.        , 0.        , 0.00246837, 0.36185009, 0.56170859],
       [0.        , 0.        , 0.00246761, 0.36454961, 0.58063977],
       [0.        , 0.        , 0.0033914 , 0.36105567, 0.59345022],
       [0.        , 0.        , 0.00215916, 0.37377194, 0.57121576],
       [0.        , 0.        , 0.00615385, 0.35630252, 0.5748413 ],
       [0.        , 0.        , 0.00400555, 0.36512684, 0.57547325],
       [0.        , 0.        , 0.00277606, 0.36231722, 0.57730924],
       [0.        , 0.        , 0.00185099, 0.36091393, 0.58509142],
       [0.        , 0.        , 0.00277521, 0.37000995, 0.56469391],
       [0.        , 0.        , 0.00185185, 0.37035398, 0.57620632]])}


Best regularization seems to be at C=1 for tfid datasets, though the score isn't much better, lets try adding a subset of char features to the tfid set

In [0]:
from sklearn.pipeline import FeatureUnion
tfid_plus_char = FeatureUnion([ #unify features from the two vectorizers
('word_vectorizer',  TfidfVectorizer(
sublinear_tf=True,
stop_words = 'english',
encoding='ISO-8859-1',
analyzer='word',
token_pattern=r'\w{1,}',
ngram_range=(1,3),
dtype=np.float32,
max_features=5000)),

('char_vectorizer', TfidfVectorizer(
sublinear_tf=True,
stop_words = 'english',
encoding='ISO-8859-1',
analyzer='char',
ngram_range=(2, 6),
dtype=np.float32,
max_features=6000))
 ])#FeatureUnion([TFID_NGram,CV_tfid_char])
x_train_tfid_full = tfid_plus_char.fit_transform(train_data_X)
x_test_tfid_full = tfid_plus_char.transform(test_data_x)

In [0]:
tfid_bow_c_w = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[1],scoring='f1', verbose=1, cv=10, n_jobs=-1).fit(x_train_tfid_full, train_data_Y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  5.5min finished


In [0]:
tfid_bow_c_w_score = tfid_bow_c_w.score(x_test_tfid_full,test_data_y)
print ("Score for TFID/NGram (1,3) w/ chars Ngram 2-6(min_df = 5): " + str(tfid_bow_c_w_score))
print(tfid_bow_c_w.scores_)

Score for TFID/NGram (1,3) w/ chars Ngram 2-6(min_df = 5): 0.611693856899573
{True: array([[0.59894266],
       [0.60635336],
       [0.61686826],
       [0.6013838 ],
       [0.61007265],
       [0.59406586],
       [0.60734186],
       [0.61362148],
       [0.6087312 ],
       [0.61296884]])}


Got some improvement here, but not alot! Lets move to the next part

Part 3: Other data about the text to look at


We're going to need to use a pipeline for this since we're merging a bunch of random features here

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import string

def get_text_length(x): # https://stackoverflow.com/q/39121104/
    return np.array([len(t) for t in x]).reshape(-1, 1)

def caps_cnt(x):
  return np.array([(sum(1 for c in t if c.isupper())) for t in x]).reshape(-1, 1)

def punc_cnt(x):
  return np.array([(sum(1 for c in t if (c in string.punctuation))) for t in x]).reshape(-1, 1)


In [0]:
tfid_cwl = Pipeline([
    ('features', FeatureUnion([
        ('word_vectorizer',  TfidfVectorizer(
            sublinear_tf=True,
            stop_words = 'english',
            encoding='ISO-8859-1',
            analyzer='word',
            token_pattern=r'\w{1,}',
            ngram_range=(1,3),
            dtype=np.float32,
            max_features=5000)),
        ('count', FunctionTransformer(get_text_length, validate=False)),
        ('char_vectorizer', TfidfVectorizer(
            sublinear_tf=True,
            stop_words = 'english',
            encoding='ISO-8859-1',
            analyzer='char',
            ngram_range=(2, 6),
            dtype=np.float32,
            max_features=6000))
        ])),
      ('classifier',LogisticRegressionCV(
            solver='liblinear',
            penalty='l1',
            Cs=[1],
            scoring='f1',
            verbose=1,
            cv=10,
            n_jobs=-1))
      ])
      
tfid_cwl.fit(train_data_X,train_data_Y)
tfid_cwl_score = skl.metrics.f1_score(tfid_cwl.predict(test_data_x),test_data_y) 
print("Score for TFID using words+chars, + length feature: " + str(tfid_cwl_score))


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.5min finished


Score for TFID using words+chars, + length feature: 0.6117844536051937


Length on it's own didn't seem to do much. Lets try adding in some stuff like Capitilization? 

In [0]:
tfid_cwlc = Pipeline([
    ('features', FeatureUnion([
        ('word_vectorizer',  TfidfVectorizer(
            sublinear_tf=True,
            stop_words = 'english',
            encoding='ISO-8859-1',
            analyzer='word',
            token_pattern=r'\w{1,}',
            ngram_range=(1,3),
            dtype=np.float32,
            max_features=5000)),
        ('count', FunctionTransformer(get_text_length, validate=False)),
        ('caps', FunctionTransformer(caps_cnt, validate=False)),
        ('char_vectorizer', TfidfVectorizer(
            sublinear_tf=True,
            stop_words = 'english',
            encoding='ISO-8859-1',
            analyzer='char',
            ngram_range=(2, 6),
            dtype=np.float32,
            max_features=6000))
        ])),
      ('classifier',LogisticRegressionCV(
            solver='liblinear',
            penalty='l1',
            Cs=[1],
            scoring='f1',
            verbose=1,
            cv=10,
            n_jobs=-1))
      ])
tfid_cwlc.fit(train_data_X,train_data_Y)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.2min finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('word_vectorizer',
                                                 TfidfVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.float32'>,
                                                                 encoding='ISO-8859-1',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=5000,
                                                                 min_df=1,
      

In [0]:
tfid_cwlc_score = tfid_cwlc.score(test_data_x,test_data_y) 
print("Score for TFID using words+chars, + length feature: " + str(tfid_cwlc_score))

Score for TFID using words+chars, + length feature: 0.6108204166255307


Not much of a change here with Caps count, either? Intersting. Most of the text data here is just raw text, no HTML or otherwise tags/indicators, One more try with punctuation count for the sake of it, but at this point I feel that these types of features are providing diminishing (if any) returns in their current form. 

In [0]:
tfid_cwlcp = Pipeline([
    ('features', FeatureUnion([
        ('word_vectorizer',  TfidfVectorizer(
            sublinear_tf=True,
            stop_words = 'english',
            encoding='ISO-8859-1',
            analyzer='word',
            token_pattern=r'\w{1,}',
            ngram_range=(1,3),
            dtype=np.float32,
            max_features=5000)),
        ('count', FunctionTransformer(get_text_length, validate=False)),
        ('caps', FunctionTransformer(caps_cnt, validate=False)),
        ('punctuation', FunctionTransformer(punc_cnt, validate=False)),
        ('char_vectorizer', TfidfVectorizer(
            sublinear_tf=True,
            stop_words = 'english',
            encoding='ISO-8859-1',
            analyzer='char',
            ngram_range=(2, 6),
            dtype=np.float32,
            max_features=6000))
        ])),
      ('classifier',LogisticRegressionCV(
            solver='liblinear',
            penalty='l1',
            Cs=[1],
            scoring='f1',
            verbose=1,
            cv=10,
            n_jobs=-1))
      ])
tfid_cwlcp.fit(train_data_X,train_data_Y)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.1min finished


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('word_vectorizer',
                                                 TfidfVectorizer(analyzer='word',
                                                                 binary=False,
                                                                 decode_error='strict',
                                                                 dtype=<class 'numpy.float32'>,
                                                                 encoding='ISO-8859-1',
                                                                 input='content',
                                                                 lowercase=True,
                                                                 max_df=1.0,
                                                                 max_features=5000,
                                                                 min_df=1,
      

In [0]:
tfid_cwlcp_score = tfid_cwlcp.score(test_data_x,test_data_y) 
print("Score for TFID using words+chars+punctuation, + length feature: " + str(tfid_cwlcp_score))

Score for TFID using words+chars+punctuation, + length feature: 0.611190605417674


Looks like punctuation counts didn't do much, as expected. Lets move on to a different method!

Part 4: Word Embeddings

Lets try a Word2Vec model using Spacey's "en_core_web" large model.

In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load(disable=["tagger", "parser", "ner"])
print("Starting vector ops...")
x_train_nlp = np.vstack([nlp(d).vector for d in train_data_X])
x_test_nlp = np.vstack([nlp(d).vector for d in test_data_x])
print("starting LR fit...")
nlp_lr = LogisticRegressionCV(solver='liblinear',penalty='l1',Cs=[.01,1,10,100],scoring='f1', verbose=1, cv=10, n_jobs=-1).fit(x_train_nlp, train_data_Y)

nlp_lr_score = nlp_lr.score(x_test_nlp,test_data_y)
print ("Score for TFID/NGram (1,3) w/ chars Ngram 2-6(min_df = 5): " + str(nlp_lr_score))
print(nlp_lr.scores_)


Starting vector ops...
starting LR fit...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 114.5min finished


Score for TFID/NGram (1,3) w/ chars Ngram 2-6(min_df = 5): 0.5110815602836879
{True: array([[0.46743738, 0.51223521, 0.51300301, 0.51281584],
       [0.45258453, 0.50284143, 0.50311584, 0.50334525],
       [0.46692382, 0.51463481, 0.51594678, 0.51612903],
       [0.46873499, 0.50938731, 0.50973937, 0.50946329],
       [0.45782897, 0.50982192, 0.51113759, 0.5110011 ],
       [0.45844608, 0.50569016, 0.50468148, 0.50486328],
       [0.4647778 , 0.5144067 , 0.51513757, 0.5152324 ],
       [0.45415612, 0.50344131, 0.50316311, 0.50288858],
       [0.46458978, 0.50938043, 0.51082488, 0.51136882],
       [0.45800194, 0.50362352, 0.50407248, 0.50452758]])}


So, this took quite a while to run. I wasn't sure of what sort of C values would work best, but it seems like this really didn't do so hot, scoring worse than BoW. This might've been due to something that I did, but I wasn't expecting the performance to tank this much